<a href="https://colab.research.google.com/github/jmhuer/Fall2021MusicResearch/blob/main/HumanFeedbackAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import re
import subprocess


def download_gdrive(id, print_stout=True):
  coomand = 'gdown https://drive.google.com/uc?id={}'.format(id)
  returned_value = subprocess.run(coomand, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  if print_stout: print(returned_value.stdout.decode("utf-8"))
  else: print("Download Complete")

In [2]:
download_gdrive("1TMo-SiXUvMDUSIKPRIhpD1zrp_6gOw44", print_stout=True)

Downloading...
From: https://drive.google.com/uc?id=1TMo-SiXUvMDUSIKPRIhpD1zrp_6gOw44
To: /content/responses.zip
100%|██████████| 480k/480k [00:00<00:00, 51.9MB/s]



In [3]:
!unzip responses.zip
!pip install tika


Archive:  responses.zip
   creating: responses/
  inflating: responses/website email-8.pdf  
  inflating: __MACOSX/responses/._website email-8.pdf  
  inflating: responses/website email-9.pdf  
  inflating: __MACOSX/responses/._website email-9.pdf  
  inflating: responses/website email-16.pdf  
  inflating: __MACOSX/responses/._website email-16.pdf  
  inflating: responses/website email-17.pdf  
  inflating: __MACOSX/responses/._website email-17.pdf  
  inflating: responses/website email-15.pdf  
  inflating: __MACOSX/responses/._website email-15.pdf  
  inflating: responses/website email-14.pdf  
  inflating: __MACOSX/responses/._website email-14.pdf  
  inflating: responses/website email-10.pdf  
  inflating: __MACOSX/responses/._website email-10.pdf  
  inflating: responses/website email-11.pdf  
  inflating: __MACOSX/responses/._website email-11.pdf  
  inflating: responses/website email-13.pdf  
  inflating: __MACOSX/responses/._website email-13.pdf  
  inflating: responses/websit

In [4]:
#!/usr/bin/python3

import os, glob
from tika import parser 
from pandas import DataFrame
import pandas as pd

# What file extension to find, and where to look from
ext = "*.pdf"
PATH = "responses"

# Find all the files with that extension
files = []
for dirpath, dirnames, filenames in os.walk(PATH):
    files += glob.glob(os.path.join(dirpath, ext))

# Create a Pandas Dataframe to hold the filenames and the text
df = DataFrame(columns=("filename","text"))

# Process each file in turn, parsing with Tika and storing in the dataframe
for idx, filename in enumerate(files):
   data = parser.from_file(filename)
   text = data["content"]
   df.loc[idx] = [filename, text]

# For debugging, print what we found
# print(df.loc[0][1])
# indx = df.loc[0][1]



for index, row in df.iterrows():
    indx = row["text"].find("FROM") + len("FROM")
    if indx - len("FROM")!=-1:
      group_number = row["text"][indx:indx+1]
      print(row["filename"], group_number)
    # if group_number==" ":
    #     print(row["text"])


2022-04-01 11:23:04,484 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2022-04-01 11:23:05,132 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2022-04-01 11:23:05,458 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...
2022-04-01 11:23:10,470 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


responses/website email-3.pdf 3
responses/website email-10.pdf 1
responses/website email-16.pdf 2
responses/website email-20.pdf 1
responses/website email-15.pdf 2
responses/website email.pdf 2
responses/website email-14.pdf 2
responses/website email-4.pdf 2
responses/website email-17.pdf 3
responses/website email-5.pdf 2
responses/website email-9.pdf 1
responses/website email-21.pdf 3
responses/website email-7.pdf 1
responses/website email-2.pdf 3
responses/website email-11.pdf 1
responses/website email-22.pdf 2
responses/website email-6.pdf 2
responses/website email-13.pdf 3
responses/website email-19.pdf 3
responses/website email-12.pdf 1
responses/website email-8.pdf 1
responses/website email-18.pdf 2


In [89]:
import re
  
# Function to extract all the numbers from the given string
def getNumbers(str):
    array = re.findall(r'[0-9]+', str)
    return array
import statistics

class SurveyData:
  def __init__(self, raw_dataframe):
      self.group_map = {1:{"Q1_1.0": [1,2,3,4,5],
          "Q1_1.5": [6,7,8,9,10],
          "Q1_0.5": [11,12,13,14,15],
          "Q1_0.7": [16,17,18,19,20],
          "Q1_1.9": [21,22,23,24,25],
          "Q2_1.9": [26, 27, 28, 29, 30],
          "Q2_1.5": [31,32,33,34,35],
          "Q2_1.0": [36, 37, 38 ,39, 40],
          "Q2_0.7": [41,42,43,44,45],
          "Q2_0.5": [46,47,48,49,50],
          "Q3_0.5": [51],
          "Q3_1.0": [52,53,56],
          "Q3_1.9": [54],
          "Q3_1.5": [55],
          "Q4_Vary": [59,57],
          "Q4_Human": [60,58]},
          2:{"Q1_1.0": [1,2,3,4,5],
          "Q1_1.9": [6,7,8,9,10],
          "Q1_0.5": [11,12,13,14,15],
          "Q1_1.5": [16,17,18,19,20],
          "Q1_0.7": [21,22,23,24,25],
          "Q2_0.5": [26, 27, 28, 29, 30],
          "Q2_0.7": [31,32,33,34,35],
          "Q2_1.0": [36, 37, 38 ,39, 40],
          "Q2_1.5": [41,42,43,44,45],
          "Q2_1.9": [46,47,48,49,50],
          "Q3_0.5": [51],
          "Q3_1.0": [56,53,52],
          "Q3_1.9": [54],
          "Q3_1.5": [55],
          "Q4_Vary": [57,59],
          "Q4_Human": [58,60]},
          3:{"Q1_0.7": [1,2,3,4,5],
          "Q1_0.5": [6,7,8,9,10],
          "Q1_1.9": [11,12,13,14,15],
          "Q1_1.5": [16,17,18,19,20],
          "Q1_1.0": [21,22,23,24,25],
          "Q2_1.9": [26, 27, 28, 29, 30],
          "Q2_0.7": [31,32,33,34,35],
          "Q2_1.5": [36, 37, 38 ,39, 40],
          "Q2_1.0": [41,42,43,44,45],
          "Q2_0.5": [46,47,48,49,50],
          "Q3_1.9": [52],
          "Q3_1.5": [53,55],
          "Q3_1.0": [56,54,51],
          "Q4_Vary": [59,57],
          "Q4_Human": [60,58]}
          }
      self.raw_dataframe = raw_dataframe
      self.d = {'Q1_0.5':[], 'Q1_0.7':[], 'Q1_1.0':[], 'Q1_1.5':[],'Q1_1.9':[], 
                'Q2_0.5':[], 'Q2_0.7':[], 'Q2_1.0':[], 'Q2_1.5':[],'Q2_1.9':[],
                'Q3_0.5':[], 'Q3_0.7':[], 'Q3_1.0':[], 'Q3_1.5':[],'Q3_1.9':[],
                'Q4_Human':[],'Q4_Vary':[]}
      self.cleaned_data_dic = self.make_cleaned_data_dic()
  def val2musicExample(self, val, group):
      for Q in self.group_map[group]:
          if val in self.group_map[group][Q]: 
              return Q
  def number2score(self, number, seqIndx):
      if seqIndx < 10: 
        score = int(number) - seqIndx*5 
      else: 
        score = 1
      return score
  def make_cleaned_data_dic(self):
      for index, row in self.raw_dataframe.iterrows():
          indx = row["text"].find("FROM") + len("FROM")
          if indx - len("FROM")!=-1:
              group_number = int(row["text"][indx:indx+1])
          ##DO STUFF 
          vals = []
          raw = getNumbers(row["text"])[-15:]
    
          for seqIndx in range(len(raw)):
            c = self.val2musicExample(int(raw[seqIndx]),1)
            score = self.number2score(raw[seqIndx], seqIndx)
            if score > 0: self.d[c].append(self.number2score(raw[seqIndx], seqIndx))

          # ##add a 0 for every row not appended
          # for Q in self.d.keys():
          #   print(Q[0:2])
          #   if Q[0:2]=="Q3" and Q[0:2]=="Q4" and len(self.d[Q])!=index+1: 
          #       self.d[Q].append(0)
      print(self.d)
      # df = pd.DataFrame(data=self.d)
      return df
  def print_summary(self):
      for Q in self.d.keys():
        if len(self.d[Q]) > 1: 
          print(Q + " count: " + str(len(self.d[Q])) +  " " + str(statistics.mean(self.d[Q])))

In [90]:
s = SurveyData(df)
s.print_summary()

{'Q1_0.5': [5, 3, 1, 2, 2, 3, 1, 4, 3, 1, 3, 1, 3, 5, 4, 2, 5, 4, 3, 5, 2], 'Q1_0.7': [4, 2, 4, 3, 4, 3, 4, 3, 5, 1, 4, 2, 5, 5, 5, 4, 4, 4, 2, 4, 5], 'Q1_1.0': [2, 2, 3, 1, 5, 4, 5, 2, 2, 1, 5, 2, 4, 2, 1, 5, 3, 3, 4, 4, 2, 4], 'Q1_1.5': [1, 2, 1, 2, 3, 2, 3, 1, 2, 3, 1, 5, 1, 2, 5, 5, 1, 3, 3, 1, 1], 'Q1_1.9': [3, 3, 2, 4, 3, 3, 2, 4, 4, 4, 3, 4, 4, 3, 4, 1, 2, 3, 2, 4, 3], 'Q2_0.5': [1, 1, 5, 1, 5, 5, 4, 4, 2, 5, 1, 2, 1, 2, 1, 5, 4, 1, 2, 2, 1, 5], 'Q2_0.7': [3, 2, 4, 2, 5, 4, 5, 5, 3, 3, 1, 3, 2, 1, 3, 5, 5, 4, 3, 1, 2, 2], 'Q2_1.0': [4, 4, 3, 3, 4, 4, 3, 4, 2, 4, 4, 4, 1, 5, 2, 5, 4, 5, 4, 1, 3, 4], 'Q2_1.5': [2, 2, 2, 4, 3, 3, 2, 3, 1, 2, 3, 3, 3, 3, 5, 4, 3, 2, 1, 4, 4, 3], 'Q2_1.9': [5, 2, 1, 3, 3, 2, 1, 1, 4, 1, 3, 2, 4, 2, 4, 3, 1, 3, 2, 3, 5, 1], 'Q3_0.5': [1, 1, 1, 1, 1, 1, 1, 1], 'Q3_0.7': [], 'Q3_1.0': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'Q3_1.5': [1, 1, 1, 1, 1, 1, 1, 1], 'Q3_